IAMSI -- 2021-2022
--------


# TME 09 : Règles d'association

<font color="RED" size="+1">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms**

- KRISNI Almehdi (3800519)
- ARICHANDRA Santhos (3802651)

## Présentation

### But de ce TME

Ce TME a pour but de réaliser une implémentation *intuitive* de l'algorithme **apriori** afin de la comparer à une version efficace de cet algorithme, puis à une implémentation de l'algorithme **fp-growth**.


#### Contenu de l'archive téléchargée

Une fois détarée, l'archive **tme-09.tgz** crée le répertoire <code>tme09</code> contenant :
- ce notebook que vous lisez en ce moment et que vous allez compléter.
- un répertoire <code>dataset</code> contenant des bases d'exemples pour tester les algorithmes
- deux fichiers exécutables (Linux): <code>apriori32</code> et <code>fpgrowth32</code> (ces 2 programmes ont été développés par Christian Borgelt, plus d'infos sur sa page web  http://www.borgelt.net/software.html). Ces programmes seront utilisés en fin de séance. Si vous n'êtes pas sous Linux, vous pouvez trouver une version pour votre OS ici:
    - <code>apriori</code> : https://borgelt.net/apriori.html
    - <code>fpgrowth</code> : https://borgelt.net/fpgrowth.html



#### Compte-rendu de la séance

Le compte-rendu de ce TME se compose de ce notebook complété par les réponses aux questions posées.

Ce compte-rendu est à poster : 
- à l'issue de la séance, un premier envoi doit être **obligatoirement** fait avec ce que vous avez réalisé
- si nécessaire, un complément peut être envoyé **au plus tard avant le début de la prochaine séance**.


<font color="RED">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : *Nom1_Nom2.ipynb* 
- *Nom1* et *Nom2* : noms des membres du binôme
- ne pas compresser ou faire une archive: envoyer le notebook tel quel, éventuellement, si vous avez d'autres fichiers à envoyer, les  joindre au message.

In [6]:
# Vérification de la version de Python utilisée:
import sys
sys.path # le path doit contenir python3.5 ou une version supérieure

['d:\\Almehdi\\Documents\\UPMC\\M1_DAC\\S2\\IAMSI\\TME\\TME9',
 'C:\\Users\\Almehdi\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip',
 'C:\\Users\\Almehdi\\AppData\\Local\\Programs\\Python\\Python39\\DLLs',
 'C:\\Users\\Almehdi\\AppData\\Local\\Programs\\Python\\Python39\\lib',
 'C:\\Users\\Almehdi\\AppData\\Local\\Programs\\Python\\Python39',
 '',
 'C:\\Users\\Almehdi\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'd:\\almehdi\\documents\\upmc\\m1_dac\\s2\\footballgame-ml\\gameengine',
 'C:\\Users\\Almehdi\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages',
 'C:\\Users\\Almehdi\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\win32',
 'C:\\Users\\Almehdi\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Almehdi\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Almehdi\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\IPython\\extensions',
 'C:\

## Traitement d'une base d'apprentissage


### Chargement de la base

On commence par travailler sur la base exemple du fichier <code>exemple-1.txt</code> (fourni dans le répertoire datasets). 

Ce fichier contient une transaction par ligne. Chaque transaction est composée d'un groupe d'items séparés par un espace.

On peut charger en Python ce fichier par la commande suivante (le répertoire datasets doit se trouver dans le répertoire courant) :

In [7]:
import csv
with open('datasets/exemple-1.txt', 'r') as fichier:
    lecteur = csv.reader(fichier, delimiter=' ')
    i = 0
    for ligne in lecteur:
        i += 1
        print('ligne',i,':',ligne)

ligne 1 : ['a', 'b', 'c']
ligne 2 : ['a', 'd', 'e']
ligne 3 : ['b', 'c', 'd']
ligne 4 : ['a', 'b', 'c', 'd']
ligne 5 : ['b', 'c']
ligne 6 : ['a', 'b', 'd']
ligne 7 : ['d', 'e']
ligne 8 : ['a', 'b', 'c', 'd']
ligne 9 : ['c', 'd', 'e']
ligne 10 : ['a', 'b', 'c']


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>chargeBase</code> qui prend en argument un nom de fichier, respectant le format énoncé plus haut, le lit et rend un dictionnaire dont les clés sont les numéros de ligne (une transaction) et les valeurs associées les itemsets correspondants représentés sous forme d'ensembles Python (des <code>set()</code> donc).

Ici, il est plus intéressant de représenter un itemset comme un ensemble Python plutôt que comme une liste. Il est ainsi plus facile de réaliser des comparaisons d'ensembles ou des ajouts d'éléments.

Dans le reste de ce document, on appelle **BASE** un dictionnaire de ce type.

In [8]:
def chargeBase(filename, delimiteur=' ') :
    # Création du dictionnaire
    d = dict()

    # On ouvre le fichier et on met à jour le dictionnaire
    with open(filename, 'r') as fichier:
        lecteur = csv.reader(fichier, delimiter=delimiteur)
        i = 0
        for ligne in lecteur:
            i += 1
            d[i] = set(ligne)

    # On retourne le dictionnaire
    return d

In [9]:
print("Résultat du chargement de 'exemple-1.txt', on obtient : ")
Base1 = chargeBase('datasets/exemple-1.txt')
Base1

Résultat du chargement de 'exemple-1.txt', on obtient : 


{1: {'a', 'b', 'c'},
 2: {'a', 'd', 'e'},
 3: {'b', 'c', 'd'},
 4: {'a', 'b', 'c', 'd'},
 5: {'b', 'c'},
 6: {'a', 'b', 'd'},
 7: {'d', 'e'},
 8: {'a', 'b', 'c', 'd'},
 9: {'c', 'd', 'e'},
 10: {'a', 'b', 'c'}}

On utilise la variable **Base1** dans la suite pour faire référence à cette base.

## Itemsets et support

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>noms_items</code> qui prend en argument une BASE et rend l'ensemble des items qui composent cette base.

In [10]:
def noms_items(base) :
    # On crée un ensemble
    s = set()

    # On itère sur la base et on met à jour l'ensemble
    for i in base.items() :
        s = s.union(i[1])

    # On retourne l'ensemble
    return s

In [11]:
print("Pour la BASE précédente :")
noms_items(Base1)

Pour la BASE précédente :


{'a', 'b', 'c', 'd', 'e'}

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>singletons</code> qui prend en argument une BASE et rend la liste des itemsets de taille 1 obtenus à partir de cette base.

*Remarque*: attention, ici on utilise une **liste** pour stocker les itemsets (un itemset est un ensemble Python) car il n'est pas possible en Python de créer des ensembles d'ensembles.


In [12]:
def singletons(base) :
    # On crée une liste pour sauvegarder les singletons (itemsets de taille 1)
    l = []

    # On cherche les singletons
    for i in noms_items(base) :
        j = set([i])
        l.append(j)

    # On retourne la liste
    return l

In [13]:
print("Exemple: singletons(Base1) rend :")
singletons(Base1)

Exemple: singletons(Base1) rend :


[{'b'}, {'d'}, {'c'}, {'e'}, {'a'}]

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>comptage</code> qui, pour une BASE et un itemset donnés, rend le nombre de transactions de BASE qui contiennent cet itemset.

In [14]:
def comptage(base, itemset) :
    # On crée un compteur
    c = 0

    # On compte le nombre de transactions qui contiennent l'itemset
    for i in base.items() :
        if (itemset.issubset(i[1])) :
            c += 1

    # On retourne le compteur
    return c

In [15]:
print("Comptage de l'itemset {'a','b','c'} dans la base précédente : ")
print("comptage(Base1,{'a','b','c'}) rend la valeur "+str(comptage(Base1,{'a','b','c'})))

Comptage de l'itemset {'a','b','c'} dans la base précédente : 
comptage(Base1,{'a','b','c'}) rend la valeur 4


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>support</code> qui, pour une BASE et un itemset donnés, rend le support de cet itemset dans la BASE.

In [16]:
def support(base, itemset) :
    # On retourne la valeur du support (nombr / nombre d'items dans la base)
    return comptage(base, itemset) / len(base)

In [17]:
print("Support de l'itemset {'a','b','c'} dans la base précédente : " \
      +str(support(Base1,{'a','b','c'})))

Support de l'itemset {'a','b','c'} dans la base précédente : 0.4


## Implémentation de l'algorithme a-priori

Dans cette partie, une implémentation de la partie de construction des itemsets fréquents de l'algorithme a-priori est réalisée. On ne s'intéresse pas dans cette question à la génération des règles d'association (mais cela peut être fait en complément).

Votre programme doit pouvoir s'appliquer aux bases fournies dans le répertoire *datasets* (éventuellement, sur au moins les 10 premiers exemples de la base mushrooms).

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>apriori_gen</code> qui prend en argument une liste d'itemsets de même longueur $k$, applique l'algorithme apriori-gen pour rendre la liste des itemsets candidats de longueurs $k+1$.


In [18]:
def apriori_gen(itemlist) :
    # On crée la liste des itemsets candidats de longueur (k + 1)
    l = []

    # On itère sur la liste
    for i in range(len(itemlist)) :
        for j in itemlist[i:] :
            if itemlist[i] != j :
                tmp = itemlist[i].union(j)
                if (len(tmp) == len(j) + 1) and (tmp not in l) :
                    l.append(tmp)

    # On effectue une vérification sur les ensembles trouvés
    fl = l.copy() # Liste finale allant être retournée
    for cset in l :
        for j in cset :
            tmp = cset.copy()
            tmp.remove(j)
            if (tmp not in itemlist) and (cset in fl) :
                fl.pop(fl.index(cset))

    # On retourne la liste
    return fl

In [19]:
print("Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend ")
print(apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]))

print("\nExemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend ")
print(apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]))

print("\nExemple: apriori_gen([{'a','b','d'}, {'b','c','d'}]) rend ")
print(apriori_gen([{'a','b','d'}, {'b','c','d'}]))

Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend 
[{'b', 'a'}, {'a', 'c'}, {'d', 'a'}, {'b', 'c'}, {'b', 'd'}, {'d', 'c'}]

Exemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend 
[{'b', 'd', 'a'}, {'b', 'd', 'c'}]

Exemple: apriori_gen([{'a','b','d'}, {'b','c','d'}]) rend 
[]


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>apriori</code> qui prend en argument une BASE et une valeur réelle comprise entre 0 et 1, et qui rend une liste de tuples dont le premier élément et un itemset trouvé et le deuxième élément est la valeur de support correspondante.

In [20]:
def apriori(base, minsup) :
    # On récupère la liste originale des itemsets à étudier (donc les singletons)
    l = singletons(base)
    size = len(base)

    # Liste résultat allant contenir les tuples
    res = []

    # On itère sur la liste et on met à jour
    while l :
        # Création de liste pour la sauvegarde des supports et des comptages
        supportList = []
        comptList = []

        # On sauvegarde les comptages dans l'ordre des itemset
        for itemset in l :
            comptList.append(comptage(base, itemset))

        # On met à jour les supports
        tmpItemset = [] # Liste allant servir à générer les nouveaux itemsets à étudier

        # On itère sur le dictionnaire
        for i in range(len(l)) :
            if (comptList[i] / size) >= minsup :
                tmpItemset.append(l[i])
                supportList.append(comptList[i] / size)

        # On met à jour l
        l = apriori_gen(tmpItemset)

        # On sauvegarde les résultats
        for i in range(len(tmpItemset)) :
            res.append((tmpItemset[i], supportList[i]))

    # On retourne le résultat
    return res

In [21]:
print("Exemple: apriori(Base1, 0.3]) rend ")
print(apriori(Base1,0.3))

Exemple: apriori(Base1, 0.3]) rend 
[({'b'}, 0.7), ({'d'}, 0.7), ({'c'}, 0.7), ({'e'}, 0.3), ({'a'}, 0.6), ({'b', 'd'}, 0.4), ({'b', 'c'}, 0.6), ({'b', 'a'}, 0.5), ({'d', 'c'}, 0.4), ({'e', 'd'}, 0.3), ({'d', 'a'}, 0.4), ({'a', 'c'}, 0.4), ({'b', 'd', 'c'}, 0.3), ({'b', 'd', 'a'}, 0.3), ({'b', 'a', 'c'}, 0.4)]


## Expérimentations

### Vérification avec la base du TD

Utiliser la fonction <code>apriori</code> avec la base de transactions du TD afin de vérifier que les itemsets obtenus sont corrects.

In [22]:
# On reproduit la base étudiée pendant le TD
BaseTD = {
    1 : {'A', 'B', 'D', 'E'},
    2 : {'B', 'C', 'D'},
    3 : {'A', 'B', 'D', 'E'},
    4 : {'A', 'C', 'D', 'E'},
    5 : {'B', 'C', 'D', 'E'},
    6 : {'B', 'D', 'E'},
    7 : {'C', 'D'},
    8 : {'A', 'B', 'C'},
    9 : {'A', 'D', 'E'},
    10 : {'B', 'D'}
}

In [23]:
print("Exemple: apriori(BaseTD, 0.3) rend ")
print(apriori(BaseTD,0.3))

Exemple: apriori(BaseTD, 0.3) rend 
[({'B'}, 0.7), ({'E'}, 0.6), ({'A'}, 0.5), ({'D'}, 0.9), ({'C'}, 0.5), ({'B', 'E'}, 0.4), ({'B', 'A'}, 0.3), ({'B', 'D'}, 0.6), ({'B', 'C'}, 0.3), ({'E', 'A'}, 0.4), ({'E', 'D'}, 0.6), ({'A', 'D'}, 0.4), ({'C', 'D'}, 0.4), ({'B', 'E', 'D'}, 0.4), ({'E', 'A', 'D'}, 0.4)]


### Comparaisons avec les implémentations apriori et fpgrowth

<font color="RED" size="+1">**[Q]**</font>  Tester votre fonction <code>apriori</code> sur les 3 bases données dans datasets et confronter les résultats obtenus avec ceux fournis par l'exécutable <code>apriori</code>.

In [24]:
print("Résultat du chargement de 'mushrooms.txt', on obtient : ")
Base_mushrooms = chargeBase('datasets/mushrooms.txt',delimiteur=',')
#Base_mushrooms
print(len(Base_mushrooms),' transactions.')

Résultat du chargement de 'mushrooms.txt', on obtient : 
8416  transactions.


In [25]:
print("Pour la BASE précédente :")
noms_items(Base_mushrooms)
print("Exemple: singletons(Base_mushrooms) rend :")
singletons(Base_mushrooms)

Pour la BASE précédente :
Exemple: singletons(Base_mushrooms) rend :


[{'KNOBBED'},
 {'CLOSE'},
 {'SUNKEN'},
 {'EVANESCENT'},
 {'FREE'},
 {'FIBROUS'},
 {'FOUL'},
 {'ENLARGING'},
 {'WASTE'},
 {'CONVEX'},
 {'ANISE'},
 {'FISHY'},
 {'CHOCOLATE'},
 {'BELL'},
 {'NO'},
 {'LEAVES'},
 {'FLARING'},
 {'SILKY'},
 {'CLUB'},
 {'SCALY'},
 {'GROOVES'},
 {'GRASSES'},
 {'ALMOND'},
 {'ABUNDANT'},
 {'CREOSOTE'},
 {'CINNAMON'},
 {'LARGE'},
 {'ORANGE'},
 {'CONICAL'},
 {'MEADOWS'},
 {'PUNGENT'},
 {'PARTIAL'},
 {'?'},
 {'WOODS'},
 {'NARROW'},
 {'GRAY'},
 {'PATHS'},
 {'ONE'},
 {'EQUAL'},
 {'BUFF'},
 {'SPICY'},
 {'WHITE'},
 {'MUSTY'},
 {'BRUISES'},
 {'PURPLE'},
 {'ATTACHED'},
 {'POISONOUS'},
 {'NUMEROUS'},
 {'SEVERAL'},
 {'TWO'},
 {'BROWN'},
 {'PENDANT'},
 {'BROAD'},
 {'ROOTED'},
 {'BLACK'},
 {'EDIBLE'},
 {'PINK'},
 {'SOLITARY'},
 {'GREEN'},
 {'FLAT'},
 {'CLUSTERED'},
 {'TAPERING'},
 {'RED'},
 {'BULBOUS'},
 {'URBAN'},
 {'NONE'},
 {'SMOOTH'},
 {'CROWDED'},
 {'SCATTERED'},
 {'YELLOW'}]

In [26]:
print(apriori(Base_mushrooms,0.9))

[({'FREE'}, 0.9743346007604563), ({'PARTIAL'}, 1.0), ({'ONE'}, 0.9230038022813688), ({'WHITE'}, 0.9771863117870723), ({'PARTIAL', 'FREE'}, 0.9743346007604563), ({'ONE', 'FREE'}, 0.9001901140684411), ({'WHITE', 'FREE'}, 0.9743346007604563), ({'PARTIAL', 'ONE'}, 0.9230038022813688), ({'WHITE', 'PARTIAL'}, 0.9771863117870723), ({'WHITE', 'ONE'}, 0.9001901140684411), ({'PARTIAL', 'ONE', 'FREE'}, 0.9001901140684411), ({'WHITE', 'PARTIAL', 'FREE'}, 0.9743346007604563), ({'WHITE', 'ONE', 'FREE'}, 0.9001901140684411), ({'WHITE', 'PARTIAL', 'ONE'}, 0.9001901140684411), ({'WHITE', 'ONE', 'PARTIAL', 'FREE'}, 0.9001901140684411)]


In [27]:
# On effectue un meilleur affichage en copiant le résultat
[({'ONE'}, 0.9230038022813688), ({'FREE'}, 0.9743346007604563), ({'WHITE'}, 0.9771863117870723), ({'PARTIAL'}, 1.0), ({'ONE', 'FREE'}, 0.9001901140684411), ({'ONE', 'WHITE'}, 0.9001901140684411), ({'ONE', 'PARTIAL'}, 0.9230038022813688), ({'WHITE', 'FREE'}, 0.9743346007604563), ({'PARTIAL', 'FREE'}, 0.9743346007604563), ({'WHITE', 'PARTIAL'}, 0.9771863117870723), ({'ONE', 'FREE', 'WHITE'}, 0.9001901140684411), ({'ONE', 'FREE', 'PARTIAL'}, 0.9001901140684411), ({'PARTIAL', 'ONE', 'WHITE'}, 0.9001901140684411), ({'WHITE', 'FREE', 'PARTIAL'}, 0.9743346007604563), ({'ONE', 'FREE', 'WHITE', 'PARTIAL'}, 0.9001901140684411)]

[({'ONE'}, 0.9230038022813688),
 ({'FREE'}, 0.9743346007604563),
 ({'WHITE'}, 0.9771863117870723),
 ({'PARTIAL'}, 1.0),
 ({'FREE', 'ONE'}, 0.9001901140684411),
 ({'ONE', 'WHITE'}, 0.9001901140684411),
 ({'ONE', 'PARTIAL'}, 0.9230038022813688),
 ({'FREE', 'WHITE'}, 0.9743346007604563),
 ({'FREE', 'PARTIAL'}, 0.9743346007604563),
 ({'PARTIAL', 'WHITE'}, 0.9771863117870723),
 ({'FREE', 'ONE', 'WHITE'}, 0.9001901140684411),
 ({'FREE', 'ONE', 'PARTIAL'}, 0.9001901140684411),
 ({'ONE', 'PARTIAL', 'WHITE'}, 0.9001901140684411),
 ({'FREE', 'PARTIAL', 'WHITE'}, 0.9743346007604563),
 ({'FREE', 'ONE', 'PARTIAL', 'WHITE'}, 0.9001901140684411)]

In [28]:
print("Résultat du chargement de 'titanic-red.csv', on obtient : ")
Base_titanic = chargeBase('datasets/titanic-red.csv',delimiteur=',')
print(len(Base_titanic),' transactions.')

Résultat du chargement de 'titanic-red.csv', on obtient : 
741  transactions.


In [29]:
# décommenter les lignes suivantes:

print("Pour la BASE Titanic :")
noms_items(Base_titanic)

print("Exemple: singletons(Base_titanic) rend :")
singletons(Base_titanic)

Pour la BASE Titanic :
Exemple: singletons(Base_titanic) rend :


[{'LondonMontrealPQ'},
 {'MerrillWI'},
 {'EnglandOglesbyIL'},
 {'NewYorkNYBriarcliffManorNY'},
 {'SpainHavanaCuba'},
 {'BaselSwitzerland'},
 {'CoCorkIrelandCharlestownMA'},
 {'CoClareIrelandWashingtonDC'},
 {'Greece'},
 {'IsleworthEngland'},
 {'CornwallHancockMI'},
 {'ColumbusOH'},
 {'EastOrangeNJ'},
 {'BerneSwitzerlandCentralCityIA'},
 {'FondduLacWI'},
 {'AughnacliffCoLongfordIrelandNewYorkNY'},
 {'CoLongfordIrelandNewYorkNY'},
 {'BromsgroveEnglandMontrealPQ'},
 {'male'},
 {'SouthamptonNewYorkNY'},
 {'AuburnNY'},
 {'ClevedonEngland'},
 {'ItalyPhiladelphiaPA'},
 {'Argentina'},
 {'LondonBrooklynNY'},
 {'StockholmSwedenNewYork'},
 {'IndiaRapidCitySD'},
 {'Portugal'},
 {'ToftaSwedenJolietIL'},
 {'EnglandSanFranciscoCA'},
 {'AustraliaFingalND'},
 {'WinnipegMN'},
 {'StockholmSwedenWashingtonDC'},
 {'NewYorkNYIthacaNY'},
 {'BrynMawrPA'},
 {'DorchesterMA'},
 {'TranvikFinlandNewYork'},
 {'StreathamSurrey'},
 {'GuernseyElizabethNJ'},
 {'DevonportEngland'},
 {'GenevaSwitzerlandRadnorPA'},
 {'New

In [30]:
Lres = apriori(Base_titanic,0.2)

# tri sur les valeurs de supports:
print(sorted(Lres, key=lambda data: data[1]))

# tri sur la taille des itemsets
print(sorted(Lres, key=lambda data: len(data[0])))

for t in sorted(Lres, key=lambda data: data[1],reverse = True):
    if len(t[0])>1:
        print(t)


[({'alive', 'Southampton', 'female'}, 0.203778677462888), ({'2nd', 'male'}, 0.20647773279352227), ({'1st', 'Southampton'}, 0.20917678812415655), ({'1st', 'male'}, 0.21997300944669365), ({'1st', 'alive'}, 0.23076923076923078), ({'Cherbourg'}, 0.23751686909581646), ({'Southampton', 'female'}, 0.2483130904183536), ({'3rd'}, 0.27125506072874495), ({'alive', 'Southampton'}, 0.2982456140350877), ({'2nd', 'Southampton'}, 0.30634278002699056), ({'alive', 'female'}, 0.32388663967611336), ({'2nd'}, 0.3481781376518219), ({'dead', 'male', 'Southampton'}, 0.3684210526315789), ({'1st'}, 0.3792172739541161), ({'female'}, 0.38866396761133604), ({'dead', 'Southampton'}, 0.41295546558704455), ({'male', 'Southampton'}, 0.46288798920377866), ({'alive'}, 0.4642375168690958), ({'dead', 'male'}, 0.46963562753036436), ({'dead'}, 0.5344129554655871), ({'male'}, 0.6099865047233468), ({'Southampton'}, 0.7112010796221323)]
[({'male'}, 0.6099865047233468), ({'dead'}, 0.5344129554655871), ({'3rd'}, 0.27125506072874

In [31]:
# Nouvel affichage pour faciliter la compréhention
print("Itemsets et valeurs de support pour Titanic :")
[({'Southampton', 'female', 'alive'}, 0.203778677462888), ({'male', '2nd'}, 0.20647773279352227), ({'1st', 'Southampton'}, 0.20917678812415655), ({'1st', 'male'}, 0.21997300944669365), ({'1st', 'alive'}, 0.23076923076923078), ({'Cherbourg'}, 0.23751686909581646), ({'Southampton', 'female'}, 0.2483130904183536), ({'3rd'}, 0.27125506072874495), ({'Southampton', 'alive'}, 0.2982456140350877), ({'Southampton', '2nd'}, 0.30634278002699056), ({'female', 'alive'}, 0.32388663967611336), ({'2nd'}, 0.3481781376518219), ({'dead', 'Southampton', 'male'}, 0.3684210526315789), ({'1st'}, 0.3792172739541161), ({'female'}, 0.38866396761133604), ({'dead', 'Southampton'}, 0.41295546558704455), ({'male', 'Southampton'}, 0.46288798920377866), ({'alive'}, 0.4642375168690958), ({'dead', 'male'}, 0.46963562753036436), ({'dead'}, 0.5344129554655871), ({'male'}, 0.6099865047233468), ({'Southampton'}, 0.7112010796221323)]

Itemsets et valeurs de support pour Titanic :


[({'Southampton', 'alive', 'female'}, 0.203778677462888),
 ({'2nd', 'male'}, 0.20647773279352227),
 ({'1st', 'Southampton'}, 0.20917678812415655),
 ({'1st', 'male'}, 0.21997300944669365),
 ({'1st', 'alive'}, 0.23076923076923078),
 ({'Cherbourg'}, 0.23751686909581646),
 ({'Southampton', 'female'}, 0.2483130904183536),
 ({'3rd'}, 0.27125506072874495),
 ({'Southampton', 'alive'}, 0.2982456140350877),
 ({'2nd', 'Southampton'}, 0.30634278002699056),
 ({'alive', 'female'}, 0.32388663967611336),
 ({'2nd'}, 0.3481781376518219),
 ({'Southampton', 'dead', 'male'}, 0.3684210526315789),
 ({'1st'}, 0.3792172739541161),
 ({'female'}, 0.38866396761133604),
 ({'Southampton', 'dead'}, 0.41295546558704455),
 ({'Southampton', 'male'}, 0.46288798920377866),
 ({'alive'}, 0.4642375168690958),
 ({'dead', 'male'}, 0.46963562753036436),
 ({'dead'}, 0.5344129554655871),
 ({'male'}, 0.6099865047233468),
 ({'Southampton'}, 0.7112010796221323)]

In [32]:
# Nouvel affichage pour faciliter la compréhension
print("Itemsets et valeurs de supports pour Titanic triés en fonction de la taille des itemsets :")
[({'alive'}, 0.4642375168690958), ({'Southampton'}, 0.7112010796221323), ({'Cherbourg'}, 0.23751686909581646), ({'3rd'}, 0.27125506072874495), ({'2nd'}, 0.3481781376518219), ({'dead'}, 0.5344129554655871), ({'female'}, 0.38866396761133604), ({'1st'}, 0.3792172739541161), ({'male'}, 0.6099865047233468), ({'Southampton', 'alive'}, 0.2982456140350877), ({'female', 'alive'}, 0.32388663967611336), ({'1st', 'alive'}, 0.23076923076923078), ({'Southampton', '2nd'}, 0.30634278002699056), ({'dead', 'Southampton'}, 0.41295546558704455), ({'Southampton', 'female'}, 0.2483130904183536), ({'1st', 'Southampton'}, 0.20917678812415655), ({'male', 'Southampton'}, 0.46288798920377866), ({'male', '2nd'}, 0.20647773279352227), ({'dead', 'male'}, 0.46963562753036436), ({'1st', 'male'}, 0.21997300944669365), ({'Southampton', 'female', 'alive'}, 0.203778677462888), ({'dead', 'Southampton', 'male'}, 0.3684210526315789)]

Itemsets et valeurs de supports pour Titanic triés en fonction de la taille des itemsets :


[({'alive'}, 0.4642375168690958),
 ({'Southampton'}, 0.7112010796221323),
 ({'Cherbourg'}, 0.23751686909581646),
 ({'3rd'}, 0.27125506072874495),
 ({'2nd'}, 0.3481781376518219),
 ({'dead'}, 0.5344129554655871),
 ({'female'}, 0.38866396761133604),
 ({'1st'}, 0.3792172739541161),
 ({'male'}, 0.6099865047233468),
 ({'Southampton', 'alive'}, 0.2982456140350877),
 ({'alive', 'female'}, 0.32388663967611336),
 ({'1st', 'alive'}, 0.23076923076923078),
 ({'2nd', 'Southampton'}, 0.30634278002699056),
 ({'Southampton', 'dead'}, 0.41295546558704455),
 ({'Southampton', 'female'}, 0.2483130904183536),
 ({'1st', 'Southampton'}, 0.20917678812415655),
 ({'Southampton', 'male'}, 0.46288798920377866),
 ({'2nd', 'male'}, 0.20647773279352227),
 ({'dead', 'male'}, 0.46963562753036436),
 ({'1st', 'male'}, 0.21997300944669365),
 ({'Southampton', 'alive', 'female'}, 0.203778677462888),
 ({'Southampton', 'dead', 'male'}, 0.3684210526315789)]

<font color="RED" size="+1">**[Q]**</font>  Utiliser l'éxécutable <code>apriori</code> pour générer des règles d'association (voir les différentes options en annexe) sur les différentes bases de données fournies. Tester différents seuils (support et confiance). 

In [33]:
import time
startTime = time.time()
!apriori -trs50m2n5 datasets/exemple-1.txt results/resultat-apriori.txt
endTime = time.time() - startTime

print("Résultat de l'utilisation de apriori (",endTime, "secondes ) :")
with open("results/resultat-apriori.txt", "r") as f :
    lines = f.readlines()
    for l in lines :
        print(l)

Résultat de l'utilisation de apriori ( 0.02050471305847168 secondes ) :
c <- a b (50, 80)

b <- a (60, 83.3333)

c <- b (70, 85.7143)

b <- c (70, 85.7143)



apriori - find frequent item sets with the apriori algorithm
version 6.29 (2020.06.15)        (c) 1996-2020   Christian Borgelt
reading datasets/exemple-1.txt ... [5 item(s), 10 transaction(s)] done [0.00s].
filtering, sorting and recoding items ... [4 item(s)] done [0.00s].
sorting and reducing transactions ... [8/10 transaction(s)] done [0.00s].
building transaction tree ... [12 node(s)] done [0.00s].
checking subsets of size 1 2 3 done [0.00s].
writing results/resultat-apriori.txt ... [4 rule(s)] done [0.00s].


<font color="RED" size="+1">**[Q]**</font>   Ajouter les deux mesures d'intérêt vues en TD (lift et RR) et afficher leurs valeurs pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.


<font color="RED" size="+1">**[Q]**</font>  Ajouter les deux mesures d'intérêt suivantes et afficher leurs valeurs
  pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.
  
Par exemple:

\begin{align}
  \mbox{Interest}(X \longrightarrow Y) & =  \frac{P(X,Y)}{P(X)}P(Y) \nonumber\\
 \mbox{IS}(X \longrightarrow Y) & =  \frac{P(X,Y)}{\sqrt{P(X)P(Y)}} \nonumber
\end{align}

### Utilisation de fp-growth

<font color="RED" size="+1">**[Q]**</font>   Utiliser <code>fpgrowth</code> pour générer des règles d'association sur les différentes
  bases de données. Comparer avec les résultats obtenus dans la section précédente. En  particulier, comparer les temps d'éxecution.

In [34]:
startTime = time.time()
!fpgrowth -trs50m2n5 datasets/exemple-1.txt results/resultat-fpgrowth.txt
endTime = time.time() - startTime

print("Résultat de l'utilisation de fpgrowth (",endTime, "secondes ) :")
with open("results/resultat-fpgrowth.txt", "r") as f :
    lines = f.readlines()
    for l in lines :
        print(l)

Résultat de l'utilisation de fpgrowth ( 0.021503925323486328 secondes ) :
b <- c (70, 85.7143)

c <- b (70, 85.7143)

b <- a (60, 83.3333)

c <- a b (50, 80)



fpgrowth - find frequent item sets with the fpgrowth algorithm
version 6.19 (2020.06.15)        (c) 2004-2020   Christian Borgelt
reading datasets/exemple-1.txt ... [5 item(s), 10 transaction(s)] done [0.00s].
filtering, sorting and recoding items ... [4 item(s)] done [0.00s].
sorting and reducing transactions ... [8/10 transaction(s)] done [0.00s].
finding frequent item set(s) ... done [0.00s].
writing results/resultat-fpgrowth.txt ... [4 rule(s)] done [0.00s].


On remarque que les temps d'exécution sont similaires, que l'on utilise apriori ou fpgrowth.

<font color="RED" size="+1">**[Q]**</font>  En essayant différentes valeurs de seuil, tester la génération de règles d'association intéressantes sur les bases de données fournies.

## Annexes: utilisation des exécutables *apriori* et *fpgrowth*

La documentation complète de ces programmes est disponible aux URL suivantes:
- pour *apriori*: http://www.borgelt.net/doc/apriori/apriori.html
- pour *fp-growth*: http://www.borgelt.net/doc/fpgrowth/fpgrowth.html


Leur format d'utilisation général est:

        ./<programme> [options] infile [outfile]

(où <<code>programme</code>> est soit *apriori*, soit *fpgrowth*).

Les options de base de ces 2 programmes sont:
- sans argument: génération des itemsets fréquents (argument {<code>-ts</code> activé par défaut)
- *tr*: pour obtenir les règles d'associations
- *s*: pour fournir une valeur minimale de support. Le support est ici donné en valeur absolue ($n_{AB}$) et non pas en valeur relative ($\frac{n_{AB}}{n}$).
- *m*: pour fournir un nombre minimum d'items dans un itemset
- *n*: pour fournir un nombre maximum d'items dans un itemset


Par exemple (commandes lancées dans le répertoire père du répertoire *datasets/*):

        ./apriori -trs50m2n5 datasets/exemple-1.txt fichier-resultat.out  
        ./fpgrowth -trs50m2n5 datasets/mushrooms.txt fichier-resultat.out  


In [39]:
# Utilisation de apriori et fpgrowth sur la base mushrooms

!apriori -trs50m2n5 datasets/mushrooms.txt results/resultat-mushrooms-apriori.txt

print("Résultat de l'utilisation de apriori sur la base mushrooms :")
with open("results/resultat-mushrooms-apriori.txt", "r") as f :
    lines = f.readlines()
    for l in lines[:5] :
        print(l)

!fpgrowth -trs50m2n5 datasets/mushrooms.txt results/resultat-mushrooms-fpgrowth.txt

print("Résultat de l'utilisation de fpgrowth sur la base mushrooms :")
with open("results/resultat-mushrooms-fpgrowth.txt", "r") as f :
    lines = f.readlines()
    for l in lines[:5] :
        print(l)

Résultat de l'utilisation de apriori sur la base mushrooms :
NONE <- EDIBLE SMOOTH PARTIAL (51.0694, 81.3867)

NONE <- EDIBLE SMOOTH (51.0694, 81.3867)

NONE <- EDIBLE FREE WHITE PARTIAL (51.0456, 81.378)

NONE <- EDIBLE FREE WHITE (51.0456, 81.378)

NONE <- EDIBLE FREE PARTIAL (51.0456, 81.378)



apriori - find frequent item sets with the apriori algorithm
version 6.29 (2020.06.15)        (c) 1996-2020   Christian Borgelt
reading datasets/mushrooms.txt ... [70 item(s), 8416 transaction(s)] done [0.01s].
filtering, sorting and recoding items ... [20 item(s)] done [0.00s].
sorting and reducing transactions ... [198/8416 transaction(s)] done [0.00s].
building transaction tree ... [450 node(s)] done [0.00s].
checking subsets of size 1 2 3 4 5 done [0.00s].
writing results/resultat-mushrooms-apriori.txt ... [420 rule(s)] done [0.00s].


Résultat de l'utilisation de fpgrowth sur la base mushrooms :
PARTIAL <- WHITE (97.7186, 100)

WHITE <- PARTIAL (100, 97.7186)

PARTIAL <- FREE (97.4335, 100)

FREE <- PARTIAL (100, 97.4335)

PARTIAL <- FREE WHITE (97.4335, 100)



fpgrowth - find frequent item sets with the fpgrowth algorithm
version 6.19 (2020.06.15)        (c) 2004-2020   Christian Borgelt
reading datasets/mushrooms.txt ... [70 item(s), 8416 transaction(s)] done [0.01s].
filtering, sorting and recoding items ... [20 item(s)] done [0.00s].
sorting and reducing transactions ... [198/8416 transaction(s)] done [0.00s].
finding frequent item set(s) ... done [0.00s].
writing results/resultat-mushrooms-fpgrowth.txt ... [420 rule(s)] done [0.00s].
